# Running Semantic Functions Inline

The [previous notebook](./02-running-prompts-from-file.ipynb)
showed how to define a semantic function using a prompt template stored on a file.

In this notebook, we'll show how to use the Semantic Kernel to define functions inline with your C# code. This can be useful in a few scenarios:

* Dynamically generating the prompt using complex rules at runtime
* Writing prompts by editing C# code instead of TXT files. 
* Easily creating demos, like this document

Prompt templates are defined using the SK template language, which allows to reference variables and functions. Read [this doc](https://aka.ms/sk/howto/configurefunction) to learn more about the design decisions for prompt templating. 

For now we'll use only the `{{$input}}` variable, and see more complex templates later.

Almost all semantic function prompts have a reference to `{{$input}}`, which is the default way
a user can import content from the kernel arguments.

Prepare a semantic kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [11]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();


Installed Packages Microsoft.SemanticKernel, 1.0.1

Let's create a semantic function used to summarize content:

In [12]:
string skPrompt = """
{{$input}}

Summarize the content above.
""";


Let's configure the prompt, e.g. allowing for some creativity and a sufficient number of tokens.

In [13]:
var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.2,
    TopP = 0.5
};


The following code prepares an instance of the template, passing in the TXT and configuration above, 
and a couple of other parameters (how to render the TXT and how the template can access other functions).

This allows to see the prompt before it's sent to AI.

In [14]:
var promptTemplateConfig = new PromptTemplateConfig(skPrompt);

var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(promptTemplateConfig);

var renderedPrompt = await promptTemplate.RenderAsync(kernel);

Console.WriteLine(renderedPrompt);




Summarize the content above.


Let's transform the prompt template into a function that the kernel can execute:

In [15]:
var summaryFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);


Set up some content to summarize, here's an extract about Demo, an ancient Greek poet, taken from [Wikipedia](https://en.wikipedia.org/wiki/Demo_(ancient_Greek_poet)).

In [16]:
var input = """
Demo (ancient Greek poet)
From Wikipedia, the free encyclopedia
Demo or Damo (Greek: Δεμώ, Δαμώ; fl. c. AD 200) was a Greek woman of the Roman period, known for a single epigram, engraved upon the Colossus of Memnon, which bears her name. She speaks of herself therein as a lyric poetess dedicated to the Muses, but nothing is known of her life.[1]
Identity
Demo was evidently Greek, as her name, a traditional epithet of Demeter, signifies. The name was relatively common in the Hellenistic world, in Egypt and elsewhere, and she cannot be further identified. The date of her visit to the Colossus of Memnon cannot be established with certainty, but internal evidence on the left leg suggests her poem was inscribed there at some point in or after AD 196.[2]
Epigram
There are a number of graffiti inscriptions on the Colossus of Memnon. Following three epigrams by Julia Balbilla, a fourth epigram, in elegiac couplets, entitled and presumably authored by "Demo" or "Damo" (the Greek inscription is difficult to read), is a dedication to the Muses.[2] The poem is traditionally published with the works of Balbilla, though the internal evidence suggests a different author.[1]
In the poem, Demo explains that Memnon has shown her special respect. In return, Demo offers the gift for poetry, as a gift to the hero. At the end of this epigram, she addresses Memnon, highlighting his divine status by recalling his strength and holiness.[2]
Demo, like Julia Balbilla, writes in the artificial and poetic Aeolic dialect. The language indicates she was knowledgeable in Homeric poetry—'bearing a pleasant gift', for example, alludes to the use of that phrase throughout the Iliad and Odyssey.[a][2] 
""";


...and run the summary function:

In [17]:
var summaryResult = await kernel.InvokeAsync(summaryFunction, new() { ["input"] = input });

Console.WriteLine(summaryResult);


Error: Microsoft.SemanticKernel.HttpOperationException: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Status: 429 (Too Many Requests)
ErrorCode: insufficient_quota

Content:
{
    "error": {
        "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.",
        "type": "insufficient_quota",
        "param": null,
        "code": "insufficient_quota"
    }
}


Headers:
Date: Sat, 25 May 2024 22:25:26 GMT
Connection: keep-alive
Vary: REDACTED
X-Request-ID: REDACTED
Strict-Transport-Security: REDACTED
CF-Cache-Status: REDACTED
Server: cloudflare
CF-RAY: REDACTED
Alt-Svc: REDACTED
Content-Type: application/json; charset=utf-8
Content-Length: 337

 ---> Azure.RequestFailedException: You exceeded you

The code above shows all the steps, to understand how the function is composed step by step. However, the kernel
includes also some helpers to achieve the same more concisely.

The same function above can be executed with less code:

In [18]:
string skPrompt = """
{{$input}}

Summarize the content above.
""";

var result = await kernel.InvokePromptAsync(skPrompt, new() { ["input"] = input });

Console.WriteLine(result);


Error: Microsoft.SemanticKernel.HttpOperationException: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Status: 429 (Too Many Requests)
ErrorCode: insufficient_quota

Content:
{
    "error": {
        "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.",
        "type": "insufficient_quota",
        "param": null,
        "code": "insufficient_quota"
    }
}


Headers:
Date: Sat, 25 May 2024 22:25:26 GMT
Connection: keep-alive
Vary: REDACTED
X-Request-ID: REDACTED
Strict-Transport-Security: REDACTED
CF-Cache-Status: REDACTED
Server: cloudflare
CF-RAY: REDACTED
Alt-Svc: REDACTED
Content-Type: application/json; charset=utf-8
Content-Length: 337

 ---> Azure.RequestFailedException: You exceeded you

Here's one more example of how to write an inline Semantic Function that gives a TLDR for a piece of text.




In [19]:
string skPrompt = @"
{{$input}}

Give me the TLDR in 5 words.
";

var textToSummarize = @"
    1) A robot may not injure a human being or, through inaction,
    allow a human being to come to harm.

    2) A robot must obey orders given it by human beings except where
    such orders would conflict with the First Law.

    3) A robot must protect its own existence as long as such protection
    does not conflict with the First or Second Law.
";

var result = await kernel.InvokePromptAsync(skPrompt, new() { ["input"] = textToSummarize });

Console.WriteLine(result);


Error: Microsoft.SemanticKernel.HttpOperationException: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Status: 429 (Too Many Requests)
ErrorCode: insufficient_quota

Content:
{
    "error": {
        "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.",
        "type": "insufficient_quota",
        "param": null,
        "code": "insufficient_quota"
    }
}


Headers:
Date: Sat, 25 May 2024 22:25:27 GMT
Connection: keep-alive
Vary: REDACTED
X-Request-ID: REDACTED
Strict-Transport-Security: REDACTED
CF-Cache-Status: REDACTED
Server: cloudflare
CF-RAY: REDACTED
Alt-Svc: REDACTED
Content-Type: application/json; charset=utf-8
Content-Length: 337

 ---> Azure.RequestFailedException: You exceeded you